In [1]:
from pyflink.datastream import StreamExecutionEnvironment, TimeCharacteristic
from pyflink.table import StreamTableEnvironment, EnvironmentSettings

from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment, EnvironmentSettings

In [2]:

# f_s_env = StreamExecutionEnvironment.get_execution_environment()
# f_s_settings = EnvironmentSettings.new_instance().use_old_planner().in_streaming_mode().build()
# f_s_t_env = StreamTableEnvironment.create(f_s_env, environment_settings=f_s_settings)



In [3]:
env = StreamExecutionEnvironment.get_execution_environment()
env_settings = EnvironmentSettings.Builder().build()
t_env = StreamTableEnvironment.create(stream_execution_environment=env, environment_settings=env_settings)
# specify connector and format jars
t_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.0/lib/flink-sql-connector-kafka-1.15.0.jar")


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [4]:
source_ddl = """
        CREATE TABLE source_table(
            tag VARCHAR,
            `value` DOUBLE
        ) WITH (
          'connector' = 'kafka',
          'topic' = 'test-json',
          'properties.bootstrap.servers' = 'localhost:9092',
          'properties.group.id' = 'test-json-group-3',
          'scan.startup.mode' = 'latest-offset',
          'format' = 'json'
        )
        """

sink_ddl = """
        CREATE TABLE sink_table(
           tag VARCHAR,
            `value` DOUBLE
        ) WITH (
          'connector' = 'kafka',
          'topic' = 'test-json-sink',
          'properties.bootstrap.servers' = 'localhost:9092',
          'format' = 'json'
        )
        """


In [ ]:

t_env.execute_sql(source_ddl).print()
t_env.execute_sql("select * from source_table").print()
t_env.execute_sql(sink_ddl)

t_env.sql_query("SELECT a FROM source_table") \
    .execute_insert("sink_table").wait()


OK
+----+--------------------------------+--------------------------------+
| op |                            tag |                          value |
+----+--------------------------------+--------------------------------+
| +I |                        sensor2 |                           34.5 |
| +I |                        sensor2 |                           34.3 |
